In [ ]:
import os
import pandas as pd
import numpy as np
np.set_printoptions(linewidth=1000)

filename = '  '
path = 'data\\' # Sia所得特征值
alldata = np.loadtxt(path)
group = np.loadtxt('data_L\\')  # 备选地震波库
group_label = np.loadtxt('data_L\\') # 分类标签 96

groupIDs = ['1','2','3']
grounds =['C','D']
storeys = ['9','15','25']
j = 0
features = []
for storey in storeys:
    for ground in grounds:
        for groupID in groupIDs:
            data = alldata[group==j]
            label = group_label[group==j]
            target_class1 = np.hstack([data[label==j*4]])
            target_axis = np.mean(target_class1,axis = 0)
            features.append(target_axis)
            j = j + 1
                    
np.savetxt('Dense_data\\',features,fmt = '%.4f')

In [ ]:
# 3个参数------->256个特征值拟合


import numpy as np
from sklearn.linear_model import LinearRegression

def Linear(y,x1,x2,x3):

    model = LinearRegression()
    X = np.vstack((x1, x2, x3))
    X = X.T
    model.fit(X, y)
    a = [model.coef_[0],model.coef_[1],model.coef_[2],model.intercept_]   
    return a

def fx_feature(paras,storey,ground,group):
    T_dict = {'9':2.646,'15':2.818,'20':3.99,'25':3.509,}
    G_dict = {'C':-1,'D':1}
    D_dict = {'1':1,'2':5,'3':9}
    
    features = []
    for para in paras:
        feature = para[0]*T_dict[storey]+para[1]*G_dict[ground]+para[2]*D_dict[group]+para[3]
        features.append(feature)
    return features
T_dict = {'9':2.646,'15':2.818,'20':3.99,'25':3.509,}
G_dict = {'C':-1,'D':1}
D_dict = {'1':1,'2':5,'3':9}
groups = ['1','2','3']
grounds =['C','D']
storeys = ['9','15','25']
T_line = []
G_line = []
D_line = []
fx_para = [] 
for storey in storeys:
    for ground in grounds:
        for group in groups:
            T_line.append(T_dict[storey])
            G_line.append(G_dict[ground])
            D_line.append(D_dict[group])
features = np.loadtxt(' ')
features = features.T
for fea in features:
    para = Linear(fea,T_line,G_line,D_line)
    fx_para.append(para)

In [ ]:
def MandS_value(file,filenumber):
    
    data = np.loadtxt(file)
    number = len(data)
    data_mean = np.mean(data,axis = 0)
    MandS = []
    for i in filenumber:
        Sample = abs(data[i] - data_mean)
        Sample_mean = np.mean(Sample)
        Sample_std = np.std(Sample)
        Sample_mands = Sample_mean + Sample_std
        MandS.append(Sample_mands)
    return MandS

def MandS(file,filenumber):
   
    data = np.loadtxt(file)
    number = len(data)
    data_mean = np.mean(data,axis = 0)
    MandS = []
    for i in range(number):
        Sample = abs(data[i] - data_mean)
        Sample_mean = np.mean(Sample)
        Sample_std = np.std(Sample)
        Sample_mands = Sample_mean + Sample_std
        MandS.append(Sample_mands)      
    MandS_sort = np.argsort(MandS)
    accuracy = []
    for k in range(4):
        target_gm = MandS_sort[int(number/4*k):int(number/4*(k+1))]
        flag = 0
        for j in filenumber:
            if j in target_gm:
                flag = flag+1
        acc = flag/len(filenumber)*100
        accuracy.append(acc)
    return accuracy

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(linewidth=1000)
# kanghao 2023年10月13日
# 本程序用于测试选波

# 绘图设置
fig = plt.figure(figsize = (10,10),facecolor = 'blue')
fig,axes = plt.subplots(6, 3,figsize = (10,9))
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.5)

filename = ''
path = 'data\\' # Sia所得特征值
alldata = np.loadtxt(path)
group = np.loadtxt('data_L\\')  # 备选地震波库
group_label = np.loadtxt('data_L\\') # 分类标签

mean7_drift = []
ARD_list  = []
features = []
target_MandS_all = []

groupIDs = ['1','2','3']
grounds =['C','D']
storeys = ['9','15','25']
j = 0
accuracy_all = []
for storey in storeys:
    for ground in grounds:
        for groupID in groupIDs:
            data = alldata[group==j]
            label = group_label[group==j]

            target_class1 = np.hstack([data[label==j*4]])      # 基准特征向量
            target_axis = np.mean(target_class1,axis = 0)
            norm_list = []
            for i in range(len(data)):
                norm = np.linalg.norm(data[i]-target_axis)
                norm_list.append(norm)

            GM_list = np.argsort(norm_list)
            target_list = GM_list[:7]
            target_MandS = MandS_value('',target_list) 
            target_MandS_all.append(target_MandS)       
            ax1 = axes[int(j/3),j%3] 
            drift = np.loadtxt('') # 结构响应
            drift_mean = np.mean(drift,axis = 0)
            for l in target_list:
                floor = np.arange(0,len(drift[l]))
                ax1.plot(floor,drift[l],color = 'black')
            ax1.plot(floor,drift_mean,color = 'red',linewidth=3)
            ax1.set_ylim(0,4)
            ax1.set_yticks([2,4])       
            j = j+1 
target_MandS_all = np.array(target_MandS_all)
target_MandS_all = target_MandS_all.reshape(-1, 1)
target_MandS_all =pd.DataFrame(target_MandS_all)
with pd.ExcelWriter('processfile'+os.sep+'out.xlsx') as writer:
     target_MandS_all.to_excel(writer, sheet_name="sheet1",startcol=0,index=0)

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(linewidth=1000)
# kanghao 2023年8月15日
# 本程序用于选波，根据特征点距离基准点的距离，选取距离最小的地震动数据。

filename = '2023122301'
path = 'data\\Sia_'+filename+'.txt' # Sia所得特征值
alldata = np.loadtxt(path)
group = np.loadtxt('data_L\\Sia_9152025_L_group.txt')
mean7_drift = []
ARD_list  = []

groupIDs = ['1','2','3']
grounds =['C','D']
storeys = ['20']

# 绘图设置
fig = plt.figure(figsize = (10,10),facecolor = 'blue')
fig,axes = plt.subplots(len(storeys)*2, 3,figsize = (14,len(storeys)*3+1))
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.5)

j = 12
accuracy_all = []
target_MandS_all = []
for storey in storeys:
    for ground in grounds:
        for groupID in groupIDs:
            data = alldata[group==j]
            target_axis = fx_feature(fx_para,storey,ground,groupID) # 计算基准特征向量
            norm_list = []
            for i in range(len(data)):
                norm = np.linalg.norm(data[i]-target_axis)
                norm_list.append(norm)
            GM_list = np.argsort(norm_list)
            target_list = GM_list[:7]
            target_list = GM_list[(len(GM_list)-7):len(GM_list)]
            target_MandS = MandS_value(' ',target_list)
            target_MandS_all.append(target_MandS)     
            ax1 = axes[int((j-12)/3),j%3] 
            drift = np.loadtxt('') # 结构响应
            drift_mean = np.mean(drift,axis = 0)
            for l in target_list:
                floor = np.arange(1,len(drift[l])+1)
                ax1.plot(floor,drift[l],color = 'black')
            ax1.plot(floor,drift_mean,color = 'red',linewidth=3)
            font1 = {'family':'Times New Roman','weight':'normal','size':15,'color':'#000000'}
            ax1.set_ylim(0,4)
            ax1.set_yticks([0,2,4])
            ax1.set_ylabel('Drift ratio (%)',font1)
            ax1.set_xlim(1,20)
            ax1.set_xticks([1,5,10,15,20])
            ax1.set_xlabel('F'+storey+ground+groupID,font1)
            j = j+1
target_MandS_all = np.array(target_MandS_all)
target_MandS_all = target_MandS_all.reshape(-1, 1)
target_MandS_all =pd.DataFrame(target_MandS_all)
with pd.ExcelWriter('processfile'+os.sep+'out.xlsx') as writer:
     target_MandS_all.to_excel(writer, sheet_name="sheet1",startcol=0,index=0)          